In [14]:
import requests
headers = {'default':{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',"Referer": "https://www.google.com"},"alternative":{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',"Referer": "https://www.bing.com"}}
from bs4 import BeautifulSoup
from unidecode import unidecode
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures as pf
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler as mms
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.linear_model import Ridge
import datetime
import jdatetime
import math
import jalali_pandas
import json

In [15]:
prices=[]
kms=[]
years=[]
city=[]
dates=[]
colors=[]
body=[]
types=[]
urls=[]

In [54]:
def divar(url):
    cities=['tehran','karaj','qazvin','tabriz']
    cities=['tehran','karaj','tabriz','isfahan']
    cities=['qazvin']
    prices=[]
    kms=[]
    years=[]
    city=[]
    dates=[]
    colors=[]
    body=[]
    types=[]
    urls=[]
    today=['دقایقی' , 'لحظاتی' , 'ساعت','نیم','ربع']
    for c in cities:
        url=f'https://divar.ir/s/{c}/{url}'
        driver = webdriver.Chrome()
        driver.get(url)
        # Get the initial HTML source
        html = driver.page_source

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Print the title of the website
        try:
            print(soup.title.text)
        except AttributeError:
                pass
        # Define a function to scroll down the page
        def scroll_down():
            # Get the height of the document
            height = driver.execute_script("return document.body.scrollHeight")
            # Scroll to the bottom of the page
            driver.execute_script(f"window.scrollTo(0, {height});")
            # Wait for the new content to load
            time.sleep(3)

        # Scroll down the page 5 times
        for i in range(1):


            # Get the updated HTML source
            html = driver.page_source

            # Parse the HTML with BeautifulSoup
            soup=((BeautifulSoup(html, "html.parser")))
            results=soup.find_all('div',class_='kt-post-card__description')
            for i in range(len(results)):
                if i%2==0:
                    kms.append(unidecode((((results[i].get_text()).replace('کیلومتر','')).strip()).replace(',','')))
                    if c=='tehran':
                        city.append('Tehran')
                    elif c=='karaj':
                        city.append('Karaj')
                    elif c=='qazvin':
                        city.append('Qazvin')
                    elif c=='isfahan':
                        city.append('Isfahan')
                    elif c=='tabriz':
                        city.append('Tabriz')
                    else:
                        city.append('N/A')
                else:
                    prices.append(unidecode((((results[i].get_text()).replace('تومان','')).strip()).replace(',','')))
            results=soup.find_all("div",class_='kt-post-card__bottom')
            for i in range(len(results)):
                if any(x in today for x in (results[i].get_text()).split()):
                    #jdatetime.date.fromgregorian(now.date())
                    dates.append(str(jdatetime.date.today()))
                elif 'دیروز' in ((results[i].get_text()).split()):
                    dates.append(str(jdatetime.date.today()-datetime.timedelta(days=1)))
                elif 'پریروز' in ((results[i].get_text()).split()):
                    dates.append(str(jdatetime.date.today()-datetime.timedelta(days=2)))
                elif 'روز' in ((results[i].get_text()).split()):
                    dates.append(str(jdatetime.date.today()-datetime.timedelta(days=(int(unidecode(results[i].get_text())[0])))))
                elif (('هفته') in ((results[i].get_text()).split())) or (('هفتهٔ') in ((results[i].get_text()).split())):
                    if (results[i].get_text())[5] == ' ':
                        dates.append(str(jdatetime.date.today()-datetime.timedelta(days=(7))))
                    else:
                        dates.append(str(jdatetime.date.today()-datetime.timedelta(days=(7*int(unidecode(results[i].get_text())[0])))))
                else:
                    if i-1!=-1:
                        dates.append(dates[i-1])
                    else:
                        dates.append(str(jdatetime.date.today()))
            results=soup.find_all("div",class_='post-card-item-af972 kt-col-6-bee95 kt-col-xxl-4-e9d46')
            for i in range(len(results)):
                a_tag=results[i].find('a')
                try:

                    urls.append('https://www.divar.ir'+a_tag['href'])
                except TypeError:
                    continue
            scroll_down()
            time.sleep(3)
        # Print the number of items on the page
        print(len(soup.find_all("div", class_="item")))

        # Close the driver
        driver.close()
    colors=[]
    body=[]
    types=[]
    years=[]
    description=[]
    for i in urls:
    #     if i==urls[1]:
    #         break
        #time.sleep(1)
        try:
            response = requests.get(i, headers=headers['default'])
            if response.status_code==200:
                pass
            elif response.status_code==404:
                colors.append('err')
                body.append('err')
                types.append('err')
                years.append('err')
                description.append('err')
                continue
            else:
                print(f'Failed. The status code is {response.status_code}\n Retrying...')
                time.sleep(2)
                response = requests.get(i, headers=headers['alternative'])
                if response.status_code==200:
                    pass
                else:
                    print(f'Failed. The status code is {response.status_code}')
        except ChunkedEncodingError:
            print('ChunkedEncodingError!/n Retrying...')
            time.sleep(2)
            response = requests.get(i, headers=headers['alternative'])
            if response.status_code==200:
                pass
            else:
                print(f'Failed. The ChunkedEncodingError Presists or may be connection error with the code {response.status_code} ')
        except InvalidChunkLength:
            print('ChunkedEncodingError!/n Retrying...')
            time.sleep(2)
            response = requests.get(i, headers=headers['alternative'])
            if response.status_code==200:
                pass
            else:
                print(f'Failed. The ChunkedEncodingError Presists or may be connection error with the code {response.status_code} ')
    #     driver = webdriver.Chrome()
    #     driver.get(f'https://www.divar.ir{i}')
    #     # Get the initial HTML source
    #     html = driver.page_source
        internal_soup=BeautifulSoup(response.content,"html.parser")
        internal_soup=str(internal_soup)
    #     #results=internal_soup.find_all('span',class_="kt-group-row-item__value")
    #     driver.close()
        try:

            #match = re.search(pattern, internal_soup)
            pass

        except IndexError :
            colors.append('err')
            body.append('err')
            types.append('err')
            years.append('err')
            description.append('err')
            continue
        pattern = r'"title":"رنگ","value":"([^"]*)"'
        match = re.search(pattern, internal_soup)
        try:
            colors.append((match.group(1)).replace('\u200c',' '))
        except AttributeError :
            colors.append('err')
            body.append('err')
            types.append('err')
            years.append('err')
            description.append('err')
            continue
        pattern = r'"title":"مدل \(سال تولید\)","value":"([^"]*)"'
        match = re.search(pattern, internal_soup)
        years.append(unidecode(match.group(1)))
        pattern = r'"title":"برند و تیپ","value":"([^"]*)"'
        match = re.search(pattern, internal_soup)
        types.append((match.group(1)))
        pattern = r'"title":"وضعیت بدنه","value":"([^"]*)"'
        match = re.search(pattern, internal_soup)
        try:
            body.append((match.group(1)).replace('\u200c',' '))
        except AttributeError:
            body.append('N/A')
        try:
            pattern = r'"DESCRIPTION_ROW","children":"([^"]*)"'
            match = re.findall(pattern, internal_soup)
            description.append(match[2])
        except AttributeError:
            description.append('N/A')
    counter=0
    for i in range(len(urls)):
        if colors[i-counter]=='err':
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            counter=counter+1
            print(1)
            continue
        if ((prices[i-counter].count('1'))>=4) or ((prices[i-counter].count('6'))>=4) or ((prices[i-counter].count('5'))>=4) or ((prices[i-counter].count('4'))>=4) or ((prices[i-counter].count('3'))>=4)or ((prices[i-counter].count('2'))>=4):
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            counter=counter+1
            print(2)
            continue
        try:
            int(years[i-counter])
        except ValueError:
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            counter=counter+1
            print(3)
            continue
        try:
            int(prices[i-counter])
        except ValueError:
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            counter=counter+1
            print(4)
            continue
        if urls[i-counter].count('حواله'):
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            print(5)
            counter=counter+1
            continue
        if description[i-counter].count('حواله'):
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            print(6)
            counter=counter+1
            continue
        if description[i-counter].count('پیش پرداخت'):
            del types[i-counter]
            del years[i-counter]
            del colors[i-counter]
            del prices[i-counter]
            del kms[i-counter]
            del body[i-counter]
            del dates[i-counter]
            del city[i-counter]
            del urls[i-counter]
            del description[i-counter]
            print(7)
            counter=counter+1
            continue
    print(len(prices),len(kms),len(dates),len(urls),len(city),len(years),len(colors),len(body),len(types),len(description))
    colors=['سفید'if x=='سفید صدفی' else x for x in colors]
    return prices,kms,years,city,dates,colors,body,types,urls
def sheypoor(url):
    prices=[]
    kms=[]
    years=[]
    city=[]
    dates=[]
    colors=[]
    body=[]
    types=[]
    urls=[]
    today=['دقایقی' , 'لحظاتی' , 'ساعاتی','نیم','ربع']
    driver = webdriver.Chrome()
    driver.get(url)
    # Get the initial HTML source
    html = driver.page_source
    soup=((BeautifulSoup(html, "html.parser")))
    # Parse the HTML with BeautifulSoup

    # Print the title of the website
    #print(soup.title.text)

    # Define a function to scroll down the page
    def scroll_down():
        # Get the height of the document
        height = driver.execute_script("return document.body.scrollHeight")
        # Scroll to the bottom of the page
        driver.execute_script(f"window.scrollTo(0, {height});")
        # Wait for the new content to load
        time.sleep(2)
    # Locate the button element by its ID
    button = driver.find_element("class name", "aUnqO")

    # Click the button
    button.click()
    # Scroll down the page 5 times
    for i in range(20):
        html = driver.page_source
        soup=((BeautifulSoup(html, "html.parser")))
        results=soup.find_all('div',class_='YPVjY')
        for i in range(len(results)):
            try:
                results[i].find_all('small',class_='-GCBo')[1].text.split()[0]
                prices.append(unidecode(results[i].find('span',class_='J5s1-').text.replace(',','')))
                urls.append(results[i].find('a')['href'])
                city.append(results[i].find_all('small',class_='-GCBo')[0].text.split('،')[0])

                if any(x in today for x in (results[i].find_all('small',class_='-GCBo')[1].text.split())):
                    #jdatetime.date.fromgregorian(now.date())
                    dates.append(str(jdatetime.date.today()))
                elif 'روز' in (results[i].find_all('small',class_='-GCBo')[1].text.split()):
                    dates.append(str(jdatetime.date.today()-datetime.timedelta(days=(int(unidecode(results[i].find_all('small',class_='-GCBo')[1].text.split()[0]))))))
                else:
                    dates.append(str(jdatetime.date.today()))

            except (AttributeError,IndexError):
                    print('error')
                    continue
        scroll_down()
        time.sleep(3)
    # Print the number of items on the page

    # Close the driver
    driver.close()
    colors=[]
    kms=[]
    years=[]
    types=[]
    bodies=[]
    for i in range(len(urls)):
        response = requests.get(urls[i], headers=headers['default'])
        if response.status_code==200:
            pass
        elif response.status_code==404:
            continue
        else:
            print(f'Failed. The status code is {response.status_code}\n Retrying...')
            time.sleep(2)
            response = requests.get(urls[i], headers=headers['alternative'])
            if response.status_code==200:
                pass
            else:
                print(f'Failed. The status code is {response.status_code}')
        soup=BeautifulSoup(response.content,"html.parser")
        soup=soup.find('div',class_='bWPjU')
        if len(unidecode(soup.find_all("p", class_="_874-x")[1].text))==4:
            years.append(unidecode(soup.find_all("p", class_="_874-x")[1].text))
            try:
                int(unidecode(soup.find_all("p", class_="_874-x")[2].text).replace(',',''))
            except ValueError:
                kms.append('100000')
                types.append((soup.find_all("p", class_="_874-x")[2].text)) 
                colors.append((soup.find_all("p", class_="_874-x")[3].text))
                bodies.append((soup.find_all("p", class_="_874-x")[6].text))
                continue
            kms.append(unidecode(soup.find_all("p", class_="_874-x")[2].text).replace(',',''))
            types.append((soup.find_all("p", class_="_874-x")[3].text)) 
            colors.append((soup.find_all("p", class_="_874-x")[4].text))
            bodies.append((soup.find_all("p", class_="_874-x")[7].text))  
        else:
            try :
                int(unidecode(soup.find_all("p", class_="_874-x")[2].text).replace(',',''))
            except ValueError:
                years.append(unidecode(soup.find_all("p", class_="_874-x")[0].text))
                kms.append('100000')
                types.append((soup.find_all("p", class_="_874-x")[1].text)) 
                colors.append((soup.find_all("p", class_="_874-x")[2].text))
                bodies.append((soup.find_all("p", class_="_874-x")[5].text))
                continue
            years.append(unidecode(soup.find_all("p", class_="_874-x")[0].text))
            kms.append(unidecode(soup.find_all("p", class_="_874-x")[1].text).replace(',',''))
            types.append((soup.find_all("p", class_="_874-x")[2].text)) 
            colors.append((soup.find_all("p", class_="_874-x")[3].text))
            bodies.append((soup.find_all("p", class_="_874-x")[6].text))
            counter=0
        for i in range(len(urls)):
            if urls[i-counter].count('حواله'):
                del types[i-counter]
                del years[i-counter]
                del colors[i-counter]
                del prices[i-counter]
                del kms[i-counter]
                del bodies[i-counter]
                del dates[i-counter]
                del city[i-counter]
                del urls[i-counter]
                counter=counter+1
                continue
            else:
                pass

In [55]:
for i in [('car/renault/talisman')]:
    divar_ads=divar(i)
    prices=prices+divar_ads[0]
    kms=kms+divar_ads[1]
    years=years+divar_ads[2]
    city=city+divar_ads[3]
    dates=dates+divar_ads[4]
    colors=colors+divar_ads[5]
    body=body+divar_ads[6]
    types=types+divar_ads[7]
    urls=urls+divar_ads[8]
    

خرید و فروش و قیمت خودرو رنو تلیسمان در قزوین | دیوار
0
1 1 1 1 1 1 1 1 1 1


In [56]:
urls

['https://www.divar.ir/v/تالیسمان-۲۰۱۷_سواری-و-وانت_قزوین__دیوار/QZNhc_BL']

In [42]:
pattern = r'"title":"رنگ","value":"([^"]*)"'
internal_soup=str(internal_soup)
match = re.search(pattern, internal_soup)
match.group(1)

'سفید'

In [53]:
a=[2,4,5]
b=[3,2,6,7]
a=a+b
a

[2, 4, 5, 3, 2, 6, 7]